In [1]:
import numpy as  np
import pandas as pd 
import matplotlib.pyplot as plt 
%matplotlib inline 



In [2]:
import re 
import nltk
nltk.download('punkt')
nltk.download('wordnet')
  
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/lakshmi_shetty/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/lakshmi_shetty/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
data=pd.read_json("http://ist.gmu.edu/~hpurohit/courses/ait690-proj-data-spring17.json")
data.head()

,DATETIME,DOCUMENT_ID,LATITUDE,LONGITUDE,MESSAGE
0,2012-10-30 22:15:41,263403828328665088,37.4777,-122.223,@Zuora wants to help @Network4Good with Hurric...
1,2012-11-01 23:11:04,264142543883739136,36.7783,-119.418,@ztrip please help spread the good word on hel...
2,2012-11-04 16:07:02,265122997348753408,10000.0000,10000.000,"#ZSwaggers @Zendaya96 did this,you should too-..."
3,2012-11-05 17:04:19,265499798952628224,10000.0000,10000.000,@Zendaya96 u have inspired me girl! So can eve...
4,2012-11-04 18:41:56,265161977662435328,10000.0000,10000.000,''@Zendaya96 let's help the Hurricane Sandy vi...


In [4]:
data.isnull().any()

DATETIME       False
DOCUMENT_ID    False
LATITUDE       False
LONGITUDE      False
MESSAGE        False
dtype: bool

In [5]:
data["LATITUDE"]=data["LATITUDE"].replace(10000.0000,0)
data["LONGITUDE"]=data["LONGITUDE"].replace(10000.0000,0)

In [6]:
mean_latitude=data["LATITUDE"].mean()
mean_latitude

22.535964570972713

In [7]:
mean_longitude=data["LONGITUDE"].mean()
mean_longitude

-46.63928404976049

In [8]:

data["LATITUDE"]=data["LATITUDE"].replace(0,mean_latitude)
data["LONGITUDE"]=data["LONGITUDE"].replace(0,mean_longitude)

In [9]:
data["LONGITUDE"].head()

0   -122.223000
1   -119.418000
2    -46.639284
3    -46.639284
4    -46.639284
Name: LONGITUDE, dtype: float64

In [10]:
data["LATITUDE"].head()

0    37.477700
1    36.778300
2    22.535965
3    22.535965
4    22.535965
Name: LATITUDE, dtype: float64

In [11]:
new_dates, new_times = zip(*[(d.date(), d.time()) for d in data["DATETIME"]])
data = data.assign(DATE=new_dates, TIME=new_times)


In [12]:
data.head()

,DATETIME,DOCUMENT_ID,LATITUDE,LONGITUDE,MESSAGE,DATE,TIME
0,2012-10-30 22:15:41,263403828328665088,37.477700,-122.223000,@Zuora wants to help @Network4Good with Hurric...,2012-10-30,22:15:41
1,2012-11-01 23:11:04,264142543883739136,36.778300,-119.418000,@ztrip please help spread the good word on hel...,2012-11-01,23:11:04
2,2012-11-04 16:07:02,265122997348753408,22.535965,-46.639284,"#ZSwaggers @Zendaya96 did this,you should too-...",2012-11-04,16:07:02
3,2012-11-05 17:04:19,265499798952628224,22.535965,-46.639284,@Zendaya96 u have inspired me girl! So can eve...,2012-11-05,17:04:19
4,2012-11-04 18:41:56,265161977662435328,22.535965,-46.639284,''@Zendaya96 let's help the Hurricane Sandy vi...,2012-11-04,18:41:56


In [13]:
def decontracted(phrase):
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [14]:

def  remove_characters(sentence):

        y=sentence.split(" ")
#          print(y)
        list2=[]
        for i in y:
            i=i.lower()
            if re.search('[^a-zA-Z0-9.$]',i):
                pass
            else:
                list2.append(i)
#         print(list2)
        x=" ".join(list2)
        return x

In [15]:
# sentence="@Zuora wants to help @Network4Good with Hurric"
# x=remove_characters(sentence)
# print(x)

In [16]:
from autocorrect import spell

In [17]:
def url(msg):
    msg= re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', msg) 
    return msg

In [18]:
def Remove_len_two_words(msg):
    msg= re.sub(r'\b\w{1,2}\b', '', msg)
    return msg


In [19]:
ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()
stwords = stopwords.words('english')
stwords.remove('not')
stwords.remove('no')

In [20]:
d=data["MESSAGE"]
d.head()

0    @Zuora wants to help @Network4Good with Hurric...
1    @ztrip please help spread the good word on hel...
2    #ZSwaggers @Zendaya96 did this,you should too-...
3    @Zendaya96 u have inspired me girl! So can eve...
4    ''@Zendaya96 let's help the Hurricane Sandy vi...
Name: MESSAGE, dtype: object

In [21]:

corpus=[]
for i in range(0,len(d)):
    msg=d[i]
    msg= decontracted(msg)
    msg=msg.lower()
    msg=remove_characters(msg)
    msg=url(msg)
    msg=word_tokenize(msg)
    msg= [j for j in msg if j.isalpha()]
    msg = [ps.stem(word) for word in msg if not word in set(stwords)]
    msg = [lemmatizer.lemmatize(word, pos= "a") for word in msg ]
    msg=[spell(k) for k in msg]
    msg=  [Remove_len_two_words(l) for l in msg]
    msg = ' '.join(msg)
    corpus.append(msg)
    

In [22]:
corpus

['want help hurricane relief text Sandi donat',
 'pleas help spread good word help victim hurricane Sandi',
 'convoy send hurricane Sandi relief',
 ' inspire everyone pleas donat hurricane Sandi convoy not  want',
 'let help hurricane Sandi donat goe relief syria',
 'help hurricane Sandi victim text convoy donat pleas',
 'help hurricane Sandi text convoy spread world',
 'help hurricane Sandi victim text convoy donat goe relief effort',
 'person discount code help hurricane relief effort',
 'help donat american red cross hurricane Sandi relief effort text recross',
 'already phone affect',
 'text donat hurricane relief fund visit',
 'send hurricane Sandi relief supply via amazon wish',
 'use Ithun support hurricane Sandi relief',
 'use Ithun support hurricane Sandi american red cross',
 'donat american red cross via Ithun money rais goe arc help hurricane Sandi survivor',
 'help hurricane Sandi relief donat jewish fever north america',
 'help text red cross',
 'help affect hurricane San

In [23]:
MSG=pd.DataFrame(corpus,columns=["MESSAGE"])

In [24]:
data = data.drop(["DOCUMENT_ID","DATETIME","MESSAGE" ],axis=1)

In [25]:
data = data.join(MSG)
data.head()

,LATITUDE,LONGITUDE,DATE,TIME,MESSAGE
0,37.477700,-122.223000,2012-10-30,22:15:41,want help hurricane relief text Sandi donat
1,36.778300,-119.418000,2012-11-01,23:11:04,pleas help spread good word help victim hurric...
2,22.535965,-46.639284,2012-11-04,16:07:02,convoy send hurricane Sandi relief
3,22.535965,-46.639284,2012-11-05,17:04:19,inspire everyone pleas donat hurricane Sandi ...
4,22.535965,-46.639284,2012-11-04,18:41:56,let help hurricane Sandi donat goe relief syria


In [26]:
data.to_csv('Data_message.csv')